# Criteo Dataset

The [Criteo 1TB Click Logs](https://ailab.criteo.com/download-criteo-1tb-click-logs-dataset/) dataset is the largest publicly available dataset for recommender systems. 
It contains ~1.3 TB of uncompressed click logs containing over four billion samples spanning 24 days. 
Each record contains 40 columns:

* **Label**: Target variable that indicates if an ad was clicked (1) or not (0).
* **I1-I13**: A total of 13 columns of integer features (mostly count features).
* **C1-C26**: A total of 26 columns of categorical features. The values of these features have been hashed onto 32 bits for anonymization purposes. 

The semantic of these features is undisclosed, some features may have missing values, the rows are chronologically ordered and the columns are **tab separated** with the following schema:
```
<label> <integer feature 1> … <integer feature 13> <categorical feature 1> … <categorical feature 26>
```

Criteo provides a separate compressed .gz data file for each day, with a size of ~15GB. In order to use Vertex AI, the data needs to be a in Google [Cloud Storage](https://cloud.google.com/storage) bucket, uncompressed. We have prepared an uncompressed version of the Criteo dataset in the `gs://workshop-datasets/criteo` publically-accessible Cloud Storage. You can use the data directly from this location. 

Alternatively, you will need to download the data files from the original Criteo website, uncompress the data files, and upload the uncompressed data file to your own Cloud Storage location. Note that each day takes ~20-30min to download and extract it.


This notebook covers:
1. Exploring TSV data files in Cloud Storage
2. Loading the TSV data files from Cloud Storage to BigQuery
3. Validating the load data in BigQuery

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PROJECT = 'merlin-on-gcp'
REGION = 'us-central1'
DATASET_GCS_LOCATION = 'gs://workshop-datasets/criteo'
BQ_DATASET_NAME = 'criteo'
BQ_TRAIN_TABLE_NAME = 'train'
BQ_VALID_TABLE_NAME = 'valid'

## 1. Explore Criteo data files

In [ ]:
! gsutil ls {DATASET_GCS_LOCATION}

In [ ]:
HEADER = ["label"]
for i in range(1, 14):
    HEADER.append(f"num_feature_{i}")
for i in range(1, 27):
    HEADER.append(f"cat_feature_{i}")

In [ ]:
sample_size = 500000

day1_dataset = pd.read_csv(
    f"{DATASET_GCS_LOCATION}/day_1",
    sep="\t",
    names=HEADER,
    nrows=sample_size,
    
)

In [ ]:
day1_dataset.head(5).T

In [ ]:
day1_dataset.label.value_counts()

In [ ]:
day1_dataset.describe()

## 2. Load data to BigQuery

### Create destination BigQuery dataset

In [ ]:
!bq --location=US mk -d \
$PROJECT:$BQ_DATASET_NAME

### Load data from Cloud Storage to BigQuery

In [ ]:
schema = []
for column in HEADER:
    if "cat" in column:
        schema.append(f"{column}:STRING")
    else:
        schema.append(f"{column}:INT64")
schema = ','.join(schema)

In [ ]:
train_files = []
for v in range(23):
    train_files.append(f'"{DATASET_GCS_LOCATION}/day_{v}"')

train_files = ','.join(train_files)

In [ ]:
!bq load \
    --source_format=CSV \
    --field_delimiter=tab \
    --autodetect \
    --replace \
    {BQ_DATASET_NAME}.{BQ_TRAIN_TABLE_NAME} \
    {train_files} \
    {schema}

In [ ]:
!bq load \
    --source_format=CSV \
    --field_delimiter=tab \
    --autodetect \
    --replace \
    {BQ_DATASET_NAME}.{BQ_VALID_TABLE_NAME} \
    {DATASET_GCS_LOCATION}/day_23 \
    {schema}

## 3. Validate the data in BigQuery

In [ ]:
%%bigquery --project {PROJECT}

SELECT COUNT(*)
FROM criteo.train 

The loaded records should be ~4.19 billion.

In [ ]:
%%bigquery --project {PROJECT}

SELECT COUNT(*)
FROM criteo.valid 

The loaded records should be ~178 million.

In [ ]:
%%bigquery --project {PROJECT}

SELECT 
    label, 
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM criteo.1tb_click_logs) * 100,2) percentage,
FROM criteo.train 
GROUP BY label

In [ ]:
%%bigquery --project {PROJECT}

SELECT *
FROM criteo.train 
LIMIT 10